In [1]:
import os
import shutil
from PIL import Image

In [2]:
data_dir = '../data'
dataset_dir = os.path.join(data_dir, 'DATASET')

print('data_dir:', dataset_dir)
print('os.listdir(dataset_dir):', os.listdir(dataset_dir))

data_dir: ../data/DATASET
os.listdir(dataset_dir): ['TRAIN', 'TEST']


In [3]:


new_images_dir = os.path.join(data_dir, 'images')

if os.path.exists(new_images_dir):
    shutil.rmtree(new_images_dir)
os.makedirs(new_images_dir)

In [4]:
def create_dir_if_not_exists(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

In [5]:
targets_images = {} # example: {'godness': ['../data/images/godness/1.jpg', '../data/DATASET/godness/2.jpg', ...], ...}


# enter in all the subdirectories of the dataset directory
for sub_dir in os.listdir(dataset_dir):
    print('sub_dir:', sub_dir)
    sub_dir_path = os.path.join(dataset_dir, sub_dir)

    # enter in all subdirectories of the sub_dir
    for sub_sub_dir in os.listdir(sub_dir_path):
        print('|_sub_sub_dir:', sub_sub_dir)
        
        if sub_sub_dir not in targets_images:
            targets_images[sub_sub_dir] = []
        
        sub_sub_dir_path = os.path.join(sub_dir_path, sub_sub_dir)
        new_sub_sub_dir = os.path.join(new_images_dir, sub_sub_dir)
        create_dir_if_not_exists(new_sub_sub_dir)
        
        # enter in all the files of the sub_sub_dir
        for file in os.listdir(sub_sub_dir_path):
            file_path = os.path.join(sub_sub_dir_path, file)
            new_file_path = os.path.join(new_sub_sub_dir, file)
            if os.path.exists(new_file_path):
                print('file already exists:', new_file_path)
                continue
            shutil.copy(file_path, new_file_path)
            targets_images[sub_sub_dir].append(new_file_path)

sub_dir: TRAIN
|_sub_sub_dir: goddess
|_sub_sub_dir: tree
|_sub_sub_dir: downdog
|_sub_sub_dir: plank
|_sub_sub_dir: warrior2
sub_dir: TEST
|_sub_sub_dir: goddess
|_sub_sub_dir: tree
|_sub_sub_dir: downdog
|_sub_sub_dir: plank
|_sub_sub_dir: warrior2


In [6]:
targets_images.keys()
count = 0
for key in targets_images.keys():
    print('key:', key)
    print('len(targets_images[key]):', len(targets_images[key]))
    print('')
    count += len(targets_images[key])

print('count:', count)

def get_extension(file_path):
    return file_path.split('.')[-1]

def get_extensions(target):
    extensions = {}
    for target in targets_images.keys():
        for image_path in targets_images[target]:
            extension = get_extension(image_path)
            if extension not in extensions:
                extensions[extension] = 0
            extensions[extension] += 1
    return extensions

key: goddess
len(targets_images[key]): 260

key: tree
len(targets_images[key]): 229

key: downdog
len(targets_images[key]): 320

key: plank
len(targets_images[key]): 381

key: warrior2
len(targets_images[key]): 361

count: 1551


In [7]:
get_extensions(targets_images)

{'jpg': 1410, 'png': 110, 'JPG': 19, 'jpeg': 10, 'bmp': 1, 'PNG': 1}

In [8]:
# renaming all extensions to lowercase
for target in targets_images.keys():
    for index, image_path in enumerate(targets_images[target]):
        extension = get_extension(image_path)
        lower_extension = extension.lower()
        new_image_path = image_path.replace(extension,lower_extension)
        os.rename(image_path,new_image_path)
        targets_images[target][index] = new_image_path



In [9]:
get_extensions(targets_images)

{'jpg': 1429, 'png': 111, 'jpeg': 10, 'bmp': 1}

In [10]:
# convert all images to jpg
for target in targets_images.keys():
    for index, image_path in enumerate(targets_images[target]):
        extension = get_extension(image_path)
        if extension == 'jpg':
            continue
        new_image_path = image_path.replace(extension, 'jpg')
        im = Image.open(image_path)
        rgb_im = im.convert("RGB")
        rgb_im.save(new_image_path)
        os.remove(image_path)
        targets_images[target][index] = new_image_path
        


/usr/lib/python3/dist-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [11]:
get_extensions(targets_images)

{'jpg': 1551}

In [12]:
total = 0
for key, value in targets_images.items():
    total += len(value)
    
for key, value in targets_images.items():
    print(key, len(value), str(round(len(value)/total,2)) + "%")
    

goddess 260 0.17%
tree 229 0.15%
downdog 320 0.21%
plank 381 0.25%
warrior2 361 0.23%


In [13]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array

2024-05-01 11:09:06.871967: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 11:09:10.485593: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 11:09:10.489193: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 11:09:15.697594: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:
images = []
labels = []

for label, filenames in targets_images.items():
    images.extend(filenames)
    labels.extend([label] * len(filenames))

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)



# Now you have your one-hot encoded labels ready for training and testing


In [15]:
# Load and preprocess images
def load_and_preprocess_images(image_paths, labels):
    X = []
    filtered_labels = []

    for i, image_path in enumerate(image_paths):
        try:

            img = load_img(image_path, target_size=(224, 224))
            img_array = img_to_array(img) / 255.0  # Normalize pixel values
            X.append(img_array)
            filtered_labels.append(labels[i])
        except OSError:
            print(f"Warning: Image {image_path} is truncated. Skipping...")
    return np.array(X), filtered_labels

train_images_data, train_labels_filtered = load_and_preprocess_images(train_images, train_labels)
test_images_data, test_labels_filtered = load_and_preprocess_images(test_images, test_labels)



In [16]:
# Use LabelEncoder to convert labels to integers
label_encoder = LabelEncoder()
train_integer_encoded = label_encoder.fit_transform(train_labels_filtered)
test_integer_encoded = label_encoder.transform(test_labels_filtered)

# Use to_categorical to one-hot encode the integer-encoded labels
train_one_hot_encoded = to_categorical(train_integer_encoded)
test_one_hot_encoded = to_categorical(test_integer_encoded)

In [17]:
# Define input shape of your images
input_shape = (224, 224, 3)  # Assuming your images are RGB and 224x224 pixels

# Create the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(targets_images), activation='softmax')  # Assuming len(data) is the number of classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()


2024-05-01 11:09:53.598725: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177209344 exceeds 10% of free system memory.
2024-05-01 11:09:53.747330: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177209344 exceeds 10% of free system memory.
2024-05-01 11:09:53.794268: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177209344 exceeds 10% of free system memory.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [18]:

# Train the model
history = model.fit(
      train_images_data, train_one_hot_encoded,
      epochs=10,
      validation_data=(test_images_data, test_one_hot_encoded))

# Evaluate the model
loss, accuracy = model.evaluate(test_images_data, test_one_hot_encoded)
print(f'Test accuracy: {accuracy}')


2024-05-01 11:09:54.175299: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 746016768 exceeds 10% of free system memory.


Epoch 1/10


2024-05-01 11:09:56.943946: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177209344 exceeds 10% of free system memory.


39/39 [==============================] - ETA: 0s - loss: 1.8048 - accuracy: 0.3398

In [ ]:
np.random.seed(20)

In [ ]:
def plot_images(datagen):
  # generate samples and plot
  for i in range(9):
    # define subplot
    plt.subplot(330 + 1 + i)
    # generate batch of images
    batch = datagen.next()
    # convert to unsigned integers for viewing
    image_ = batch[0]
    # plot raw pixel data
    plt.imshow(image_[0])
  plt.show()
